In [1]:
import sys
sys.path.append('/home/additya/NUS/src/SVM/final')


from experiment import Experiment
from config import *

import importlib
# importlib.reload(experiment)

import numpy as np

from sklearn.preprocessing import normalize
import csv, os, sys
import numpy as np
from sklearn.utils import shuffle
from random import randint
import pickle
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 25})

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM

from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import roc_curve, roc_auc_score, average_precision_score
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

In [2]:
def get_classifier(svm_type, train_pos_X, train_neg_X=np.array([])):
    
    if svm_type == 'two-class':
        train_pos_Y = np.ones(train_pos_X.shape[0])
        train_neg_Y = np.zeros(train_neg_X.shape[0])    

        train_X = np.vstack((train_pos_X, train_neg_X))
        train_Y = np.hstack((train_pos_Y, train_neg_Y))
        
        assert(np.where(train_Y == 1)[0].shape[0] == train_pos_X.shape[0])
        assert(np.where(train_Y == 0)[0].shape[0] == train_neg_X.shape[0])

#         rus = RandomUnderSampler(return_indices=True)
#         (train_X, train_Y, id_rus) = rus.fit_sample(train_X, train_Y)
#         (train_X, train_Y) = shuffle(train_X, train_Y)
        
    else:
        train_X = train_pos_X
        train_Y = np.ones(train_pos_X.shape[0])
    
    print("Number of Train Positive Samples:", train_pos_X.shape[0])
    print("Number of Train Negative Samples:", train_neg_X.shape[0])
    
    if svm_type == 'two-class':
        clf = SVC(probability=True, gamma='scale')
        clf.fit(train_X, train_Y)
    elif svm_type == 'one-class':
        clf = OneClassSVM(gamma='scale')
        clf.fit(train_X)
    
    return clf

def get_preds(clf, test_X, clf_type):
    if clf_type == 'two-class':
        return clf.predict_proba(test_X)[:, 1]
    elif clf_type == 'one-class':
        preds = clf.decision_function(test_X)
        MAX = np.max(preds)
        preds = MAX - preds
        MAX = np.max(preds)
        preds = MAX - preds
        return preds

def evaluate_preds(preds, test_Y, svm_type):
    if svm_type == 'two-class':
        fpr, tpr, thresholds = roc_curve(test_Y, preds, drop_intermediate=False)
        auc_score = np.round(roc_auc_score(test_Y, preds), 8)
        return auc_score
    elif svm_type == 'one-class':
        return np.round(average_precision_score(test_Y, preds), 8)
    else:
        print('invalid SVM')
        raise

def get_fake_list(preds):
    fake_list, idx = [], 0
    while idx < preds.shape[0]:
        idx = idx
        if preds[idx] == 0:
            fake_list.append((idx, idx))
            while preds[idx] == 0:
                (x, y) = fake_list[-1]
                fake_list[-1] = (x, idx)
                idx += 1
                if idx == preds.shape[0]:
                    break
        else:
            idx = idx + 1
    return fake_list

def get_intersection(start_A, end_A, start_B, end_B):
    if (start_A > end_B) or (start_B > end_A):
        return (-1, -1)
    (intersection_s, intersection_e) = (max(start_A, start_B), min(end_A, end_B))
    return (intersection_s, intersection_e)
    
def intersection_over_union(preds, labels):
    
    new_preds = np.ones(preds.shape[0])
    pred_fake_idx = np.where(preds < 0.6)[0]
    new_preds[pred_fake_idx] = 0
      
    union = np.unique(np.concatenate((np.where(new_preds == 0)[0], np.where(labels == 0)[0]), 0)).shape[0]
    intersection = np.intersect1d(np.where(new_preds == 0)[0], np.where(labels == 0)[0]).shape[0]
    
    return np.round(float(intersection) / union, 8)

#     fake_list = get_fake_list(new_preds)
    
#     if len(fake_list) == 0:
#         return 0
        
#     if len(fake_list) > 1:
#         (X, Y, min_changes) = (0, 0, new_preds.shape[0] + 1)
#         for st in range(len(fake_list)):
#             for en in range(st, len(fake_list)):
#                 fake_start, _ = fake_list[st]
#                 _, fake_end = fake_list[en]
#                 changes = np.where(new_preds[:fake_start] == 0)[0].shape[0] + \
#                              np.where(new_preds[fake_start:fake_end + 1] == 1)[0].shape[0] + \
#                              np.where(new_preds[fake_end + 1:] == 0)[0].shape[0]
#                 if (changes < min_changes) or ((changes == min_changes) and (fake_end - fake_start > Y - X)):
#                     min_changes = changes
#                     (X, Y) = (fake_start, fake_end)

#         new_preds[:X] = 1
#         new_preds[X:Y + 1] = 0
#         new_preds[Y + 1:] = 1

#     fake_list = get_fake_list(new_preds) 
#     assert(len(fake_list) == 1)
    
#     (pred_s, pred_e) = fake_list[0]
    
#     fake_idx = np.where(labels == 0)[0]
#     (fake_s, fake_e) = (np.min(fake_idx), np.max(fake_idx))
    
#     (intersection_s, intersection_e) = get_intersection(fake_s, fake_e, pred_s, pred_e)
#     if intersection_s == intersection_e == -1:
#         intersection = 0
#     else:
#         intersection = intersection_e - intersection_s + 1
#     union = (pred_e - pred_s + 1) + (fake_e - fake_s + 1) - intersection
    
#     return np.round(float(intersection) / union, 8)    

In [3]:
def run_experiment_new(e, svm_type, iterr=50):
    e.total_correct_added, e.total_wrong_added = 0, 0
    e.initial_auc, e.final_auc = -1, -1
    e.initial_iou, e.final_iou = -1, -1
    probability_cutoff, frq_cutoff = 0.80, 25
    e.auc_arr = []
    e.iou_arr = []
    e.iterr = iterr
    e.svm_type = svm_type
    
    frames_right = (e.frame_length - 1) // 2
    (count_A, count_B, count_C) = (e.fake_s - 1 - e.train_last_center_frame, e.fake_e - e.fake_s + 1, e.features.shape[0] - 1 - frames_right - e.fake_e - 1)
    labels_in_order = np.hstack((np.ones(count_A), np.zeros(count_B)))
    labels_in_order = np.hstack((labels_in_order, np.ones(count_C)))
    print(count_A, count_B, count_C)
    print(labels_in_order.shape[0])
    assert(labels_in_order.shape[0] == count_A + count_B + count_C)
    
    for it in range(e.iterr):
        
        print("#" * 100)
        print("Iteration:", it)

        print("# train positive:", e.train_positive.shape[0])
        print("# train negative:", e.train_negative.shape[0])
        print("# test positive:", e.test_positive.shape[0])
        print("# test negative:", e.test_negative.shape[0])

        clf = get_classifier(e.svm_type, e.train_positive, e.train_negative)
        test_X = np.vstack((e.test_positive, e.test_negative))
        test_Y = np.hstack((np.ones(e.test_positive.shape[0]), np.zeros(e.test_negative.shape[0])))
        preds = get_preds(clf, test_X, e.svm_type)
        
        auc_score = evaluate_preds(preds, test_Y, e.svm_type)
        if auc_score < 0.5:
            preds = 1 - preds
            auc_score = 1 - auc_score

        original_auc_score = auc_score
        auc_score = np.round(auc_score, 2)
        
        
        print(">>>>>AUC:", auc_score)
        
        if e.initial_auc == -1:
            e.initial_auc = auc_score
        e.final_auc = auc_score
        e.auc_arr.append(auc_score)
        
        preds_in_order = np.hstack((preds[:count_A], preds[count_A + count_C:], preds[count_A:count_A + count_C]))
        assert(preds_in_order.shape[0] == preds.shape[0])
        
#         return (preds_in_order, labels_in_order)
        
        iou_score = np.round(intersection_over_union(preds_in_order, labels_in_order), 2)        
        
        if e.initial_iou == -1:
            e.initial_iou = iou_score
        e.final_iou = iou_score
        print(">>>>>IOU:", iou_score)
        
        new_auc_score = evaluate_preds(preds_in_order, labels_in_order, e.svm_type)
        if new_auc_score < 0.5:
            new_auc_score = 1 - new_auc_score
        if e.svm_type == 'two-class' and abs(new_auc_score - original_auc_score) > 0.0001:
            print("#"*20, "LABELS ORDER ERROR!", "#"*20)
            print(new_auc_score, original_auc_score)
            
        if (it > 0) and (it % 10 == 0):
            frq_cutoff += 10
            frq_cutoff = min(frq_cutoff, 100)
            probability_cutoff += 0.01
            probability_cutoff = min(probability_cutoff, 0.9)

        all_idx = preds.argsort()[::-1]
        if e.svm_type == 'two-class':
            pos_train_idx = np.array([i for i in all_idx if i not in e.added_idx and preds[i] > probability_cutoff])
        else:
            pos_train_idx = np.array([i for i in all_idx if i not in e.added_idx])
        pos_train_idx = pos_train_idx[:min(pos_train_idx.shape[0], frq_cutoff)]

        if len(pos_train_idx) == 0:
            print("Nothing to add, breaking!")
            break

        e.train_positive = np.vstack((e.train_positive, test_X[pos_train_idx]))
        print("# positive samples added =", pos_train_idx.shape[0])
        e.added_idx = np.hstack((e.added_idx, pos_train_idx))

        CORR = (np.where(test_Y[pos_train_idx] == 1)[0].shape[0])
        WR = (np.where(test_Y[pos_train_idx] == 0)[0].shape[0])

        e.total_correct_added += CORR
        e.total_wrong_added += WR

        print("Correct Added:", CORR)
        print("Wrong Added:", WR)
        print("Total Correct Added:", e.total_correct_added)
        print("Total Wrong Added:", e.total_wrong_added)    

        print("#" * 100)

    print('>>>>>>>>>>>>>>>>>>>>AUC', e.initial_auc, e.final_auc)
    print('>>>>>>>>>>>>>>>>>>>>IOU', e.initial_iou, e.final_iou)
    return e

In [ ]:
def run_experiment(e, svm_type, iterr=100):
    e.total_correct_added, e.total_wrong_added = 0, 0
    e.initial_auc, e.final_auc = -1, -1
    e.initial_iou, e.final_iou = -1, -1
    probability_cutoff, frq_cutoff = 0.90, 25
    e.auc_arr = []
    e.iou_arr = []
    e.iterr = iterr
    e.svm_type = svm_type
    
    frames_right = (e.frame_length - 1) // 2
    (count_A, count_B, count_C) = (e.fake_s - 1 - e.train_last_center_frame, e.fake_e - e.fake_s + 1, e.features.shape[0] - 1 - frames_right - e.fake_e - 1)
    labels_in_order = np.hstack((np.ones(count_A), np.zeros(count_B)))
    labels_in_order = np.hstack((labels_in_order, np.ones(count_C)))
    print(count_A, count_B, count_C)
    print(labels_in_order.shape[0])
    assert(labels_in_order.shape[0] == count_A + count_B + count_C)
    
    for it in range(e.iterr):
        
        print("#" * 100)
        print("Iteration:", it)

        print("# train positive:", e.train_positive.shape[0])
        print("# train negative:", e.train_negative.shape[0])
        print("# test positive:", e.test_positive.shape[0])
        print("# test negative:", e.test_negative.shape[0])

        clf = get_classifier(e.svm_type, e.train_positive, e.train_negative)
        test_X = np.vstack((e.test_positive, e.test_negative))
        test_Y = np.hstack((np.ones(e.test_positive.shape[0]), np.zeros(e.test_negative.shape[0])))
        preds = get_preds(clf, test_X, e.svm_type)
        
        auc_score = evaluate_preds(preds, test_Y, e.svm_type)
        if auc_score < 0.5:
            preds = 1 - preds
            auc_score = 1 - auc_score

        original_auc_score = auc_score
        auc_score = np.round(auc_score, 2)
        
        
        print(">>>>>AUC:", auc_score)
        
        if e.initial_auc == -1:
            e.initial_auc = auc_score
        e.final_auc = auc_score
        e.auc_arr.append(auc_score)
        
        preds_in_order = np.hstack((preds[:count_A], preds[count_A + count_C:], preds[count_A:count_A + count_C]))
        assert(preds_in_order.shape[0] == preds.shape[0])
        iou_score = np.round(intersection_over_union(preds_in_order, labels_in_order), 2)        
        
#         return (preds_in_order, labels_in_order)
        
        if e.initial_iou == -1:
            e.initial_iou = iou_score
        e.final_iou = iou_score
        print(">>>>>IOU:", iou_score)
        
        new_auc_score = evaluate_preds(preds_in_order, labels_in_order, e.svm_type)
        if new_auc_score < 0.5:
            new_auc_score = 1 - new_auc_score
        if e.svm_type == 'two-class' and abs(new_auc_score - original_auc_score) > 0.0001:
            print("#"*20, "LABELS ORDER ERROR!", "#"*20)
            print(new_auc_score, original_auc_score)
        
        if (it > 0) and (it % 10 == 0):
            probability_cutoff -= 0.01
            frq_cutoff += 10
            probability_cutoff = max(probability_cutoff, 0.8)
            frq_cutoff = min(frq_cutoff, 100)

        all_idx = preds.argsort()[::-1]
        if e.svm_type == 'two-class':
            pos_train_idx = np.array([i for i in all_idx if i not in e.added_idx and preds[i] > probability_cutoff])
        else:
            pos_train_idx = np.array([i for i in all_idx if i not in e.added_idx])
        pos_train_idx = pos_train_idx[:min(pos_train_idx.shape[0], frq_cutoff)]

        if len(pos_train_idx) == 0:
            print("Nothing to add, breaking!")
            break

        e.train_positive = np.vstack((e.train_positive, test_X[pos_train_idx]))
        print("# positive samples added =", pos_train_idx.shape[0])
        e.added_idx = np.hstack((e.added_idx, pos_train_idx))

        CORR = (np.where(test_Y[pos_train_idx] == 1)[0].shape[0])
        WR = (np.where(test_Y[pos_train_idx] == 0)[0].shape[0])

        e.total_correct_added += CORR
        e.total_wrong_added += WR

        print("Correct Added:", CORR)
        print("Wrong Added:", WR)
        print("Total Correct Added:", e.total_correct_added)
        print("Total Wrong Added:", e.total_wrong_added)    

        print("#" * 100)

    print('>>>>>>>>>>>>>>>>>>>>AUC', e.initial_auc, e.final_auc)
    print('>>>>>>>>>>>>>>>>>>>>IOU', e.initial_iou, e.final_iou)
    return e

In [4]:
test_keys = [420, 421, 423, 445, 496]

In [ ]:
e_dict = {}
for key in [420]:
    e_dict[key] = {}
    for fake_type in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']:
        for fl in [50, 75, 100]:
            for s in [50, 100, 250]:
                try:
                    print("starting:", key, fake_type, fl, s)
                    e_dict[key][(fake_type, 'original', 'two-class', fl, s)] = Experiment(key, fake_type, 'original', 'original', 'two-class', fl, s)
                    e_dict[key][(fake_type, 'au_corr', 'two-class', fl, s)] = Experiment(key, fake_type, 'au', 'corr', 'two-class', fl, s)
                    e_dict[key][(fake_type, 'au_corr', 'one-class', fl, s)] = Experiment(key, fake_type, 'au', 'corr', 'one-class', fl, s)
                    print("done:", key, fake_type, fl, s)
                except:
                    print("Not done:", key, fake_type, fl, s)
                    
for key in [420]:
    for fake_type in ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']:
        for fl in [50, 75, 100]:
            for s in [50, 100, 250]:
                try:
                    print("="*20, key, fake_type, "original two-class", "="*20)
                    run_experiment(e_dict[key][(fake_type, 'original', 'two-class', fl, s)], 'two-class')
                    print("DONE ORIGINAL TWO-CLASS", key, fake_type, fl, s)
        
                    print("="*20, key, fake_type, "au_corr one-class", "="*20)
                    run_experiment(e_dict[key][(fake_type, 'au_corr', 'one-class', fl, s)], 'one-class')
                    print("DONE AU CORR ONE-CLASS", key, fake_type, fl, s)

                    print("="*20, key, fake_type, "au_corr two-class", "="*20)
                    run_experiment(e_dict[key][(fake_type, 'au_corr', 'two-class', fl, s)], 'two-class')
                    print("DONE AU CORR TWO-CLASS", key, fake_type, fl, s) 
                except:
                    print("Error", key, fake_type, fl, s)